In [1]:
import pandas as pd
import numpy as np

from fuzzywuzzy import process

### Loading Data

Loading Original Data

In [2]:
# From Scimagojr
df_academicRelevance = pd.read_csv("..\\Data_Sets\\originalData\\academic_relevance_1996-2022.csv")

# From OurWorldInData
df_HDI = pd.read_csv("..\\Data_Sets\\originalData\\human-development-index.csv")
df_migrants = pd.read_csv("..\\Data_Sets\\originalData\\migrant-stock-total.csv")
df_naturalResources = pd.read_csv("..\\Data_Sets\\originalData\\naturalResourcesWealth.csv")
df_educationQuality = pd.read_csv("..\\Data_Sets\\originalData\\education_quality_by_outcome.csv")

# From World Bank
df_arableLand = pd.read_csv("..\\Data_Sets\\originalData\\arableLand.csv", skiprows=4)
df_qualifiedLaborForce = pd.read_csv("..\\Data_Sets\\originalData\\Labor_force_with_advanced_education.csv", skiprows=4)

Loading Data processed in other notebooks

In [3]:
df_climateData = pd.read_csv("..\\Data_Sets\\processed\\addData_climateData.csv")
df_countriesSize_regions = pd.read_csv("..\\Data_Sets\\processed\\addData_countriesSize_Regions.csv")
df_fromWiki = pd.read_csv("..\\Data_Sets\\processed\\addData_fromWiki.csv")
df_economicFreedom = pd.read_csv("..\\Data_Sets\\processed\\economicData_1995-2022.csv")
df_naturalDisasters = pd.read_csv("..\\Data_Sets\\processed\\addData_disasterData.csv")

### Merging Data - Part 1 

In [4]:
# Selecting only necessary columns from our main DataSet
df_master = df_economicFreedom[list(df_economicFreedom)[0:15] +
                               ['GDP per capita (current USD)'] +
                               list(df_economicFreedom)[19:27] +
                               ['Under-5 mortality rate (per 1k live births)']]

In [5]:
df_master.columns

Index(['Country Name', 'Index Year', 'Overall Score', 'Property Rights',
       'Government Integrity', 'Judicial Effectiveness', 'Government Spending',
       'Tax Burden', 'Fiscal Health', 'Business Freedom', 'Monetary Freedom',
       'Labor Freedom', 'Financial Freedom', 'Investment Freedom',
       'Trade Freedom', 'GDP per capita (current USD)', 'Total population',
       'Land area (sq. km)', 'Gini', 'Inflation CPI', 'Real interest rate',
       'Labor force size', 'Trade (% of GDP)', 'Trade in services (% of GDP)',
       'Under-5 mortality rate (per 1k live births)'],
      dtype='object')

In [6]:
df_master = df_master.merge(df_fromWiki, how='left', on=['Country Name'])

display(df_fromWiki.columns)

Index(['Country Name', 'Borders Length (in KM)', 'Neighbouring Countries',
       'isLandLocked', 'n_accessToSea', 'Values', 'Rail Density',
       'Pctg of Rail Electrified', 'Warred Against', 'Area Size (km2)',
       'Expanded EconZone Area', 'Amount of Ports', 'Distance from Equator',
       'Majoritary Religions'],
      dtype='object')

In [7]:
df_master = df_master.merge(df_countriesSize_regions[['Country Name', 'Shape_Leng', 'Region']], how='left', on=['Country Name'])

display(df_countriesSize_regions.columns)

Index(['Country Name', 'Shape_Leng', 'Shape_Area', 'SubRegion', 'Region'], dtype='object')

In [8]:
df_master = df_master.merge(df_climateData, how='left', on=['Country Name'])

display(df_climateData.columns)

Index(['Country Name', 'Climate Type', 'Average Temperature (C)'], dtype='object')

We'll need to reshape df_naturalDisasters before merging it

In [9]:
df_naturalDisasters.rename(columns={
    "Index Year (Decade)": 'Index Year'
}, inplace=True)

# Setting the minimum year in our data [note: 'Death rates from disasters' in 1990 was already halved, to look like half a decade]
df_naturalDisasters.loc[df_naturalDisasters['Index Year'] == 1990, ['Index Year']] = 1995

# Transforming Decade Data into Yearly data
df_naturalDisasters.loc[df_naturalDisasters['Index Year'] == 1995, ['Death rates from disasters']] /= 5
df_naturalDisasters.loc[df_naturalDisasters['Index Year'] == 2000, ['Death rates from disasters']] /= 10
df_naturalDisasters.loc[df_naturalDisasters['Index Year'] == 2010, ['Death rates from disasters']] /= 10

# List of years to add
years_to_add = list(range(1996, 2022+1))
years_to_add.remove(2000)
years_to_add.remove(2010)

new_rows = []
for year in years_to_add:
    for country in df_naturalDisasters['Country Name'].unique():
        new_rows.append({
            'Country Name': country,
            'Index Year': year,
            'Death rates from disasters': np.nan
            })
        
# Combining data
df_temp = pd.DataFrame(new_rows)
df_naturalDisasters = pd.concat([df_naturalDisasters, df_temp], ignore_index=True)

# Sorting By year, then by Country
df_naturalDisasters = df_naturalDisasters.sort_values(by=['Country Name', 'Index Year']).reset_index(drop=True)

# Interpolating 'Death rates from disasters'
df_naturalDisasters['Death rates from disasters'] = df_naturalDisasters.groupby('Country Name')['Death rates from disasters'].apply(lambda group: group.ffill(axis=0)).reset_index(drop=True)

# Doing the actual merging
df_master = df_master.merge(df_naturalDisasters, how='left', on=['Country Name', 'Index Year'])

display(df_naturalDisasters.columns)

Index(['Country Name', 'Index Year', 'Death rates from disasters'], dtype='object')

## Cleaning Original DataSets before merging them into df_master

In [10]:
def fuzzySearchName(name, refNames=df_master, score_threshold = 87):
    '''
        Given a country name, returns the closest match from refNames using fuzzy search.
        If no close match is found, it returns "PLEASE FILL MANUALLY".
    '''
    
    # If the name is empty or null, return "UNKNOWN"
    if pd.isna(name) or not name.strip():
        return "UNKNOWN"

    exceptions = {
        "Kyrgyzstan": "Kyrgyz Republic",
        "Congo": "Republic of Congo",
        'Congo, Rep.': "Republic of Congo",
        'Congo, Dem. Rep.': "Democratic Republic of Congo",
        'Democratic Republic Of The Congo': 'Democratic Republic of Congo',
        "Czechia": "Czech Republic",
        "Slovakia": "Slovak Republic",
        "Macao Sar, China": "Macau",
        "Macao": "Macau",
        'Republic Of The Congo': 'Republic of Congo',
        'Swaziland': 'Eswatini',
        'Korea, Rep.': 'South Korea',
        "Korea, Dem. People'S Rep.": 'North Korea',
        'St. Vincent and the Grenadines': 'Saint Vincent and the Grenadines',
        'Lao Pdr': 'Laos',
        "Hong Kong Sar, China": "Hong Kong",
        'Cabo Verde': 'Cape Verde',
        'Egypt, Arab Rep.': 'Egypt',
        'Turkiye': 'Turkey',
        'St. Lucia': 'Saint Lucia',
        'East Timor': 'Timor-Leste'    
    }
    
    if name in exceptions:
        return exceptions[name]
    
    match, score = process.extractOne(name, refNames['Country Name'].unique())

    # If a close match is found, return the match
    if score > score_threshold:
        return match
    
    # If no close match is found, ask for manual input
    return "PLEASE FILL MANUALLY"

In [11]:
def cleanedNamesMask(df, column = str()):

    '''
        Takes a dataFrame (and its column with Country Names), and returns a mask of rows to exclude
    '''

    removeList = [
    'British Indian Ocean Territory', 'Africa', 'Europe', 'North America', 'South America', 'Asia', 'Oceania',
    'Sint Maarten (Dutch part)', 'Saint Martin (French Part)', 'Antigua and Barbuda',
    'Netherlands Antilles', 'Northern Mariana Islands', 'Saint Barthelemy',
    'Saint Helena, Ascension and Tristan da Cunha', 'Saint Kitts and Nevis',
    'Serbia-Montenegro', "Turks and Caicos Islands", 'United States Virgin Islands',
    'Wallis and Futuna', 'Yugoslavia', 'Cocos (Keeling) Islands', 'Isle Of Man', 'South Georgia And South Sandwich Islands',
    'South Sudan','United States Minor Outlying Islands','Saint Pierre And Miquelon', 'American Samoa',
    'Falkland Islands (Malvinas)', 'Virgin Islands (British)', 'Saint Martin (Dutch)', 'Saint Martin (French)',
    'Heard Island And Mcdonald Islands', 'South Georgia And The South Sandwich Islands', 'Republic Of South Sudan',
    'Somalia'
    ]

    df[column] = df[column].str.strip().str.title()
    removeList = [country.title() for country in removeList]

    mask = ~df[column].isin(removeList)

    return mask

In [12]:
def fuzzyNamesFix(df, oldName = 'Country Name', newName='New Country Name'):
    '''
        Processes a DataFrame's country names to the standard we're using in this project.
        Uses to custom functions: fuzzySearchName() and cleanedNamesMask()
    '''

    # Remove rows with problematic names
    mask = cleanedNamesMask(df, str(oldName))
    df = df[mask].copy()
    display(f'% of included countries in mask: {mask.mean()}')

    # Fuzzy Search replacement
    df[str(newName)] = df[str(oldName)].apply(fuzzySearchName)

    # Excluding "PLEASE FILL MANUALLY"
    df = df[~df[str(newName)].isin(["PLEASE FILL MANUALLY"])]

    # Checking for duplicated countries
    display('','Checking for duplicated countries')
    display(df[str(newName)].describe(include=['object']), '')

    # Casting New Country Name to Country Name
    df[str(oldName)] = df[str(newName)].copy()

    # Drop unused names column (new name)
    df.drop(columns=[str(newName)], inplace=True)

    return df

In [13]:
def interpolateMiddle(series, timeField_series, n_times=0, extrapolate=4):
    '''
    Essentially, just fill the middle values (n_times) for any given Series, and project past/future for (extrapolate) times.
    
    series: the series to interpolate
    timeField_series: corresponding time data for the series
    n_times: interpolation reach. If zero, find maximum value based on 'field'.
    extrapolate: how much time to project past "middle data"
    '''
    
    # If invalid, we'll pick a value that works in this DataSet.
    if n_times <= 0:
        n_times = int((2022-1960)/2)
    
    # Setting points to help determine 'middle' data
    earliest_time = timeField_series.loc[series.notna()].min()
    latest_time = timeField_series.loc[series.notna()].max()
    
    # If there's no non-NaN value, return the Series as-is
    if pd.isna(earliest_time) or pd.isna(latest_time):
        return series
    
    # Only interpolate what has been defined as "middle data"
    maskedtimes = (timeField_series >= earliest_time)&(timeField_series <= latest_time)    
    temp_series = series[maskedtimes]
    temp_series.interpolate(method='linear', limit=n_times, limit_direction='both', inplace=True)    
    
    # Update the original Series with interpolated middle data
    series[maskedtimes] = temp_series
    
    if extrapolate > 0:
        temp_series = series
        temp_series.interpolate(method='linear', limit=extrapolate, limit_direction='both', inplace=True)
    
        # Update the original Series (on extrapolate step)
        series = temp_series
    
    return series

def process_group(group, column, *args, **kwargs):
    timeField_series = group['Index Year']
    group[column] = interpolateMiddle(group[column], timeField_series, *args, **kwargs)
    return group

#### Academic Relevance

In [14]:
df_academicRelevance.rename(columns={'Country': 'Country Name', 'H index': 'H index (Academic Papers)'}, inplace=True)

df_academicRelevance = df_academicRelevance[['Country Name', 'H index (Academic Papers)']]

In [15]:
# Excluding non-relevant countries to our DataSet
df_academicRelevance = fuzzyNamesFix(df_academicRelevance, 'Country Name')

# Sorting
df_academicRelevance = df_academicRelevance.sort_values(by=['Country Name']).reset_index(drop=True)


'% of included countries in mask: 0.9218106995884774'

''

'Checking for duplicated countries'

count               182
unique              182
top       United States
freq                  1
Name: New Country Name, dtype: object

''

In [16]:
# Manually adding Kosovo. Source: https://xk.h-index.com/en
# Manually adding North Macedonia. Source: https://www.adscientificindex.com/?country_code=mk

df_temp = pd.DataFrame({
    'Country Name': ['Kosovo', 'North Macedonia'],
    'H index (Academic Papers)': [147, 59]
})

df_academicRelevance = pd.concat([df_academicRelevance, df_temp], ignore_index=True)


In [17]:
set_A = set(df_academicRelevance['Country Name'].unique())
set_B = set(df_master['Country Name'].unique())

display(f'Names we should exclude: {set_A - set_B}')
display(f'Names that are still missing: {set_B - set_A}')

'Names we should exclude: set()'

'Names that are still missing: set()'

In [18]:
df_master = df_master.merge(df_academicRelevance, how='left', on=['Country Name'])

display(df_academicRelevance.columns)

Index(['Country Name', 'H index (Academic Papers)'], dtype='object')

#### World Bank:

Arable Land and Qualified Work Foce

In [19]:
# Excluding non-relevant countries to our DataSet
df_arableLand = fuzzyNamesFix(df_arableLand, 'Country Name')
df_qualifiedLaborForce = fuzzyNamesFix(df_qualifiedLaborForce, 'Country Name')

'% of included countries in mask: 0.9661654135338346'

''

'Checking for duplicated countries'

count             183
unique            183
top       Afghanistan
freq                1
Name: New Country Name, dtype: object

''

'% of included countries in mask: 0.9661654135338346'

''

'Checking for duplicated countries'

count             183
unique            183
top       Afghanistan
freq                1
Name: New Country Name, dtype: object

''

We'll need to transpose the DataFrames

In [20]:
years = [str(year) for year in range(1995, 2022+1)]  # This selects the columns representing 1995 through 2022
df_arableLand = df_arableLand[['Country Name'] + years]
df_qualifiedLaborForce = df_qualifiedLaborForce[['Country Name'] + years]

In [21]:
df_arableLand = df_arableLand.melt(id_vars=["Country Name"], 
                               value_vars=list(map(str, range(1995, 2023))),
                               var_name="Index Year", 
                               value_name="Arable Land pct")

df_qualifiedLaborForce = df_qualifiedLaborForce.melt(id_vars=["Country Name"], 
                               value_vars=list(map(str, range(1995, 2023))),
                               var_name="Index Year", 
                               value_name="Qualified Labor Force pct")

In [22]:
# Converting Index Year to int
df_arableLand['Index Year'] = df_arableLand['Index Year'].astype('int')
df_qualifiedLaborForce['Index Year'] = df_qualifiedLaborForce['Index Year'].astype('int')

In [23]:
display('df_arableLand Stats')
set_A = set(df_arableLand['Country Name'].unique())
set_B = set(df_master['Country Name'].unique())

display(f'Names we should exclude: {set_A - set_B}')
display(f'Names that are still missing: {set_B - set_A}')


display(df_arableLand.shape)

'df_arableLand Stats'

'Names we should exclude: set()'

'Names that are still missing: set()'

(5124, 3)

In [24]:
display('df_qualifiedLaborForce Stats')
set_A = set(df_qualifiedLaborForce['Country Name'].unique())
set_B = set(df_master['Country Name'].unique())

display(f'Names we should exclude: {set_A - set_B}')
display(f'Names that are still missing: {set_B - set_A}')


display(df_qualifiedLaborForce.shape)

'df_qualifiedLaborForce Stats'

'Names we should exclude: set()'

'Names that are still missing: set()'

(5124, 3)

In [25]:
df_master = df_master.merge(df_arableLand, how='left', on=['Country Name', 'Index Year'])
df_master = df_master.merge(df_qualifiedLaborForce, how='left', on=['Country Name', 'Index Year'])

display(df_arableLand.columns)
display(df_qualifiedLaborForce.columns)

Index(['Country Name', 'Index Year', 'Arable Land pct'], dtype='object')

Index(['Country Name', 'Index Year', 'Qualified Labor Force pct'], dtype='object')

#### Our World in Data

In [26]:
cols_to_drop = ['Human capital', 'Produced capital', 'Net foreign assets'] # We need to nesure that we're only capturing Natural Resources Value
df_naturalResources = df_naturalResources.drop(columns=cols_to_drop)

# Now let's add every col in a new col
df_naturalResources['Natural Resources'] = df_naturalResources.iloc[:, -6:].sum(axis=1)
df_naturalResources = df_naturalResources[['Entity', 'Code', 'Year', 'Natural Resources']]

# Let's select only the years we need
df_HDI = df_HDI.loc[df_HDI['Year'] >= 1995, :]
df_migrants = df_migrants.loc[df_migrants['Year'] >= 1995, :]
df_naturalResources = df_naturalResources.loc[df_naturalResources['Year'] >= 1995, :]


In [27]:
# List of years to add | educationQuality
years_to_add = list(range(1995, 2022+1))
years_to_add.remove(2010)
years_to_add.remove(2017)
years_to_add.remove(2018)
years_to_add.remove(2020)

new_rows_educationQuality = []
for year in years_to_add:
    for country in df_educationQuality['Entity'].unique():
        new_rows_educationQuality.append({
            'Entity': country,
            'Year': year,
            'Harmonized Test Scores': np.nan
            })

# List of years to add | df_naturalResources
years_to_add = list(range(1995, 2022+1))
years_to_add.remove(1995)
years_to_add.remove(2000)
years_to_add.remove(2005)
years_to_add.remove(2010)
years_to_add.remove(2014)

new_rows_naturalResources = []
for year in years_to_add:
    for country in df_naturalResources['Entity'].unique():
        new_rows_naturalResources.append({
            'Entity': country,
            'Year': year,
            'Natural Resources': np.nan
            })

# List of years to add | Migrants
years_to_add = list(range(1995, 2022+1))
years_to_add.remove(1995)
years_to_add.remove(2000)
years_to_add.remove(2005)
years_to_add.remove(2010)
years_to_add.remove(2015)

new_rows_migrants = []
for year in years_to_add:
    for country in df_migrants['Entity'].unique():
        new_rows_migrants.append({
            'Entity': country,
            'Year': year,
            'International migrant stock, total': np.nan
            })

# List of years to add | df_HDI
years_to_add = [2022]

new_rows_HDI = []
for year in years_to_add:
    for country in df_HDI['Entity'].unique():
        new_rows_HDI.append({
            'Entity': country,
            'Year': year,
            'Human Development Index': np.nan
            })

In [28]:
# Combining data
df_temp = pd.DataFrame(new_rows_educationQuality)
df_educationQuality = pd.concat([df_educationQuality, df_temp], ignore_index=True)

df_temp = pd.DataFrame(new_rows_naturalResources)
df_naturalResources = pd.concat([df_naturalResources, df_temp], ignore_index=True)

df_temp = pd.DataFrame(new_rows_migrants)
df_migrants = pd.concat([df_migrants, df_temp], ignore_index=True)

df_temp = pd.DataFrame(new_rows_HDI)
df_HDI = pd.concat([df_HDI, df_temp], ignore_index=True)

# Sorting By year, then by Country
df_educationQuality = df_educationQuality.sort_values(by=['Entity', 'Year']).reset_index(drop=True)

df_naturalResources = df_naturalResources.sort_values(by=['Entity', 'Year']).reset_index(drop=True)

df_migrants = df_migrants.sort_values(by=['Entity', 'Year']).reset_index(drop=True)

df_HDI = df_HDI.sort_values(by=['Entity', 'Year']).reset_index(drop=True)

In [29]:
# Dropping Code column before merging
df_educationQuality = df_educationQuality.drop(columns=['Code'])
df_naturalResources = df_naturalResources.drop(columns=['Code'])
df_migrants = df_migrants.drop(columns=['Code'])
df_HDI = df_HDI.drop(columns=['Code'])

# Now let's merge them into a temporary DataFrame
df_OWinData = df_HDI.merge(df_naturalResources,how='outer',on=['Entity', 'Year'])
df_OWinData = df_OWinData.merge(df_migrants,how='outer',on=['Entity', 'Year'])
df_OWinData = df_OWinData.merge(df_educationQuality,how='outer',on=['Entity', 'Year'])

df_OWinData.rename(columns={
    'Entity': 'Country Name',
    'Year': 'Index Year',
    'International migrant stock, total': 'Migration Volume',
    'Humand Development Index': 'HDI'    
}, inplace=True)


Fixing Country Names

In [30]:
df_OWinData = fuzzyNamesFix(df_OWinData, 'Country Name')

#df_OWinData.to_csv(".\\temp.csv")

set_A = set(df_OWinData['Country Name'].unique())
set_B = set(df_master['Country Name'].unique())

display(f'Names we should exclude: {set_A - set_B}')
display(f'Names that are still missing: {set_B - set_A}')


'% of included countries in mask: 0.9358732005234841'

''

'Checking for duplicated countries'

count            5123
unique            183
top       Afghanistan
freq               28
Name: New Country Name, dtype: object

''

'Names we should exclude: set()'

'Names that are still missing: set()'

In [31]:
to_interpolate = ['Human Development Index', 'Natural Resources', 'Migration Volume', 'Harmonized Test Scores']

# Process each column in DataFrame
for col in to_interpolate:
    print(f'Interpolating {col}...')
    df_OWinData = df_OWinData.groupby(by=['Country Name']).apply(lambda group: process_group(group, col, n_times=100, extrapolate=100)).reset_index(drop=True)

Interpolating Human Development Index...
Interpolating Natural Resources...
Interpolating Migration Volume...
Interpolating Harmonized Test Scores...


In [32]:
set_A = set(df_OWinData['Country Name'].unique())
set_B = set(df_master['Country Name'].unique())

display(f'Names we should exclude: {set_A - set_B}')
display(f'Names that are still missing: {set_B - set_A}')

'Names we should exclude: set()'

'Names that are still missing: set()'

### Final merge and export

In [33]:
df_master = df_master.merge(df_OWinData,how='left',on=['Country Name', 'Index Year'])

In [34]:
df_master.to_csv(f"..\\Data_Sets\\processed\\mergedData_toClean_1995-2022.csv", index=False)